# Demo of WFST-based Word Segmentation
## for Intern3 Students
by Ye Kyaw Thu, Lab Leader, LST Lab., Myanmar  
Date: 12 July 2025

## Reference
[https://www.phontron.com/kyfd/tut1/](https://www.phontron.com/kyfd/tut1/)  

C++ နဲ့ ရေးထားတဲ့ kyfd က ကျောင်းသားတွေ installation လုပ်တဲ့အခါမှာ ခက်တာရယ် ပြီးတော့ perl code တွေထက် စာရင် လက်ရှိအချိန်မှာက Python programming နဲ့သွားတာက ပိုအဆင်ပြေတာမို့လို့ ဒီ Lab တစ်ခုလုံး run လို့ရအောင် fst_decoder.py နဲ့ preprocessing, post-task တွေအတွက် လိုအပ်တဲ့ code အားလုံးကို အချိန်ယူပြင်ဆင်ခဲ့တယ်။ သို့သော်လည်း code မှာ အမှားတချို့ ရှိနိုင်တာကို ဂရုပြုပါ။  

## Data Preparation

In [1]:
!pwd

/home/ye/exp/fst_decoder


In [36]:
!tree ./script

./script
├── char_break.py
├── evaluate.py
├── evaluate_segmentation.py
├── mk_lexicon.py
├── mk_symbol.py
└── word_vocab.py

1 directory, 6 files


In [3]:
mkdir data

In [4]:
cd data

/home/ye/exp/fst_decoder/data


### Training Data
Download myPOS Version 3.0 (notag, no punctuation) text file.  
ဒီဖိုင်ကို training data အဖြစ်ထားမယ်။

In [7]:
!wget https://raw.githubusercontent.com/ye-kyaw-thu/myPOS/refs/heads/master/corpus-ver-3.0/corpus/mypos-ver.3.0.shuf.notag.nopunc.txt

--2025-07-12 22:10:43--  https://raw.githubusercontent.com/ye-kyaw-thu/myPOS/refs/heads/master/corpus-ver-3.0/corpus/mypos-ver.3.0.shuf.notag.nopunc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7303817 (7.0M) [application/octet-stream]
Saving to: ‘mypos-ver.3.0.shuf.notag.nopunc.txt’

mypos-ver.3.0.shuf. 100%[===================>]   6.96M  24.8MB/s    in 0.3s    

2025-07-12 22:10:45 (24.8 MB/s) - ‘mypos-ver.3.0.shuf.notag.nopunc.txt’ saved [7303817/7303817]



Check filesize, content ...

In [9]:
!wc mypos-ver.3.0.shuf.notag.nopunc.txt 

  43196  510644 7303817 mypos-ver.3.0.shuf.notag.nopunc.txt


In [11]:
!head mypos-ver.3.0.shuf.notag.nopunc.txt

၁၉၆၂ ခုနှစ် ခန့်မှန်း သန်းခေါင်စာရင်း အရ လူဦးရေ ၁၁၅၉၃၁ ယောက် ရှိ သည်
လူ တိုင်း တွင် သင့်မြတ် လျော်ကန် စွာ ကန့်သတ် ထား သည့် အလုပ် လုပ် ချိန် အပြင် လစာ နှင့်တကွ အခါ ကာလ အားလျော်စွာ သတ်မှတ် ထား သည့် အလုပ် အားလပ်ရက် များ ပါဝင် သည့် အနားယူခွင့် နှင့် အားလပ်ခွင့် ခံစားပိုင်ခွင့် ရှိ သည်
ဤ နည်း ကို စစ်ယူ သော နည်း ဟု ခေါ် သည်
စာပြန်ပွဲ ဆို တာ က အာဂုံဆောင် အလွတ်ကျက် ထား တဲ့ ပိဋကတ်သုံးပုံ စာပေ တွေ ကို စာစစ် သံဃာတော်ကြီး တွေ ရဲ့ ရှေ့ မှာ အလွတ် ပြန် ပြီး ရွတ်ပြ ရ တာ ပေါ့
ဒီ မှာ ကျွန်တော့် သက်သေခံကတ် ပါ
၂ဝ ရာစု မြန်မာ့ သမိုင်း သန်းဝင်းလှိုင် ၂ဝဝ၉ ခု မေ လ ကံကော်ဝတ်ရည် စာပေ
ကျွန်တော် မျက်မှန် တစ် လက် လုပ် ချင် ပါ တယ်
ကျွန်တော် တို့ က ဒီ အမှု ရဲ့ ကြံရာပါ ကို ဖမ်းမိ ဖို့ ကြိုးစား ခဲ့ တယ်
ကလေး မီးဖွား ဖို့ ခန့်မှန်း ရက် က ဘယ်တော့ ပါ လဲ
အရိုးရှင်းဆုံး ကာဗိုဟိုက်ဒရိတ် မှာ ဂလူးကို့စ် ဂလက်တို့စ် ဖရပ်တို့စ် စသည့် မိုနိုဆက်ကရိုက် များ ဖြစ် သည်


### Open-test Data
Download open-test data  
(ဒီဖိုင်ကလည်း POS tag တွေ ဖြုတ်ထားပြီးသား)

In [12]:
!wget https://raw.githubusercontent.com/ye-kyaw-thu/myPOS/refs/heads/master/corpus-draft-ver-1.0/data/otest.nopipe.word

--2025-07-12 22:20:59--  https://raw.githubusercontent.com/ye-kyaw-thu/myPOS/refs/heads/master/corpus-draft-ver-1.0/data/otest.nopipe.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 316246 (309K) [text/plain]
Saving to: ‘otest.nopipe.word’

otest.nopipe.word   100%[===================>] 308.83K  --.-KB/s    in 0.02s   

2025-07-12 22:20:59 (16.2 MB/s) - ‘otest.nopipe.word’ saved [316246/316246]



In [13]:
!wc otest.nopipe.word

  1000  21965 316246 otest.nopipe.word


In [14]:
!head otest.nopipe.word

ဆယ် ရာခိုင်နှုန်း ဈေး လျှော့ ပေး ရင် ဝယ် မယ် ။
ယခု လ ၏ အထိမ်းအမှတ် ပန်း မှာ မြတ်လေးပန်း Pomeacoccinea ဖြစ် သည် ။
ကရင် ဗမာ အဓိကရုဏ်း သည် သူ့ အား များ စွာ ဒေါမနဿ ဖြစ် စေ ပါ သည် ။
ကျောကုန်း ၏ အလယ် သည် ဝက်ဝံ ၏ တစ်ကိုယ်လုံး တွင် အမြင့်ဆုံး အစိတ်အပိုင်း ဖြစ် သည် ။
သို့သော် ၂၀၀၄ ခုနှစ် မှ စတင် ၍ အစိုးရ ၏ နိုင်ငံ ရေး လှုပ်ရှား မှု များ ကို ကန့်သတ် သော စစ်ဆင် ရေး တစ် ရပ် အနေဖြင့် ၎င်း ရုံးခွဲ များ ကို ပြန်လည် ပိတ် စေ ခြင်း သို့မဟုတ် ပြင်းထန် သော ပိတ်ပင် တားဆီး မှု များ ကို ပြုလုပ် ခဲ့ သည် ။
အပိုအလျှံ များ ကို နိုင်ငံတော် အဖွဲ့အစည်း များ သို့ တိုက်ရိုက် ရောင်းချ သည် ။
ထို ထက် တိကျ စွာ ဖော်ပြ ဦး မည် ။
ဆရာကြီး ၏ ဇနီး ဖြစ် သူ ဒေါ်စိုး မ သေ ခင် ရွှေဥဒေါင်း စာမူ ကိစ္စ နှင့် ပတ်သက် ၍ သေတမ်းစာ ရေး ခဲ့ သည် ။
မြစ်ကြီးနား မှ ၁၁၈ မိုင် အကွာ တွင် တည်ရှိ သည် ။
ကုန်းလိပ် နှင့် ပင်လယ်လိပ် တို့ သည် ကိလိုနီယာ မျိုးစဉ် တွင် ပါဝင် လေ သည် ။


### Closed-Test Data
စောစောက download လုပ်ထားတဲ့ လေးသောင်း သုံးထောင် စာကြောင်း ဖိုင်ထဲကနေပဲ ဆွဲထုတ်ယူခဲ့တယ်။  
("shuf: write error: Broken pipe" ဆိုတဲ့ error message မတက်အောင် shuf ရဲ့ stderr ကို /dev/null အောက်ကို ပို့ထားတာ)  

In [18]:
!shuf ./mypos-ver.3.0.shuf.notag.nopunc.txt 2>/dev/null | head -n 1000 > ctest.word

In [19]:
!ls

ctest.word  mypos-ver.3.0.shuf.notag.nopunc.txt  otest.nopipe.word


In [20]:
!wc ctest.word

  1000  11373 161948 ctest.word


In [21]:
!head ctest.word

ပင်လယ် နဲ့ မျက်နှာချင်းဆိုင် တစ် ခန်း
သုံး နှစ် ပျမ်းမျှ တစ် ဦး ချင်း တစ် နှစ် စုစုပေါင်း ဝင်ငွေ အမေရိကန် ဒေါ်လာ ၇၅ဝ ထက် လျော့နည်း ရ မည် ဖြစ် သည်
ဘာသာစကား မှာ ပါရမီ ပါ တယ်
ဟင့်အင်း မ ပျက် ပါ ဘူး
ပြတင်းပေါက် အပြင် ဘက် ကို ကြည့် လိုက် တဲ့ အခါ နှင်း တွေ ကျ နေ ခဲ့ တယ်
သူ အိမ် မှာ မ ရှိ ဘူး
မြောက်အမေရိက တိုက် တွင် တွေ့ ရ သော ဝက်ဝံ မျိုး များ အနက် ဝက်ဝံမည်း သည် ဝက်ဝံ မျိုး တွင် အတွေ့ ရ အများဆုံး အမျိုး ဖြစ် သည်
ကျွန်တော် ခင်ဗျား ရဲ့ ကားမောင်း လိုင်စင် အမည် လိပ်စာ နဲ့ ဖုန်း နံပါတ် တို့ ကို လည်း တချက် ကြည့် ချင် ပါ တယ်
အခန်း ရောက် သော်
နောက် အပတ် ကျွန်တော် တို့ Suzhou ကို သွား ကြ မလို့


## Preparing a Lexicon

In [23]:
!python ../script/word_vocab.py -help

usage: word_vocab.py [-h] [--input FILE] [--output FILE]

Build word vocabulary from input text

options:
  -h, --help     show this help message and exit
  --input FILE   Input text file (default: stdin)
  --output FILE  Output vocabulary file (default: stdout)


In [24]:
!python ../script/word_vocab.py < ./mypos-ver.3.0.shuf.notag.nopunc.txt > train.vocab

In [25]:
!wc train.vocab

 24797  24797 618714 train.vocab


In [26]:
!head train.vocab

%
'
''
*
+
.
..
...
....
0


In [27]:
!tail train.vocab

つ
つなみ
ビルマ人
ミャンマー
新加坡
日本語
昭南島
昭和の時代に得た南の島
缅甸
﻿တင်ပို့


In [28]:
!shuf train.vocab | head

ဝတ်စရာ
တစ်ဆယ့်ကိုး
သံဓိဌာန်ချမှတ်
အရိုအသေ
တကမ္ဘာလုံး
အရပ်ရပ်
RayOzzie
အချိန်တို
SavetheChildren
စစ်ဆေးမှု
shuf: write error: Broken pipe


## Make a FST Format Lexicon
အထက်က ဆွဲထုတ်ခဲ့တဲ့ word တွေကို FST format အဖြစ် ပြောင်းမယ်။  
အဲဒီအတွက် ရေးထားတဲ့ Python code ကိုလည်း လေ့လာကြည့်ပါ။  

In [39]:
!cat ./script/mk_lexicon.py

"""
Create lexicon FST from word list.
Written by Ye Kyaw thu, LU Lab., Myanmar.
Last updated: 11 July 2025
Usage:
    python mk_lexicon.py < train.vocab > lexicon.txt
"""

#!/usr/bin/env python3
import sys
import argparse
from collections import defaultdict

def main():
    parser = argparse.ArgumentParser(description='Create lexicon FST from word list')
    parser.add_argument('--input', metavar='FILE',
                       help='Input word list file (default: stdin)',
                       type=argparse.FileType('r', encoding='utf-8'),
                       default=sys.stdin)
    parser.add_argument('--output', metavar='FILE',
                       help='Output FST file (default: stdout)',
                       type=argparse.FileType('w', encoding='utf-8'),
                       default=sys.stdout)
    
    args = parser.parse_args()

    newstate = 1
    
    try:
        for line in args.input:
            line = line.strip()
            if not line:
                continu

In [144]:
cd ./data/

/home/ye/exp/fst_decoder/data


In [145]:
!python ../script/mk_lexicon.py < train.vocab > lexicon.txt

In [146]:
!head -n 20 ./lexicon.txt

0 1 % % 1
1 0 <w> <eps>
0 2 ' ' 1
2 0 <w> <eps>
0 3 ' '' 1
3 4 ' <eps>
4 0 <w> <eps>
0 5 * * 1
5 0 <w> <eps>
0 6 + + 1
6 0 <w> <eps>
0 7 . . 1
7 0 <w> <eps>
0 8 . .. 1
8 9 . <eps>
9 0 <w> <eps>
0 10 . ... 1
10 11 . <eps>
11 12 . <eps>
12 0 <w> <eps>


## Make Symbol Files
အရင်ဆုံး input လုပ်မယ့် ဖိုင်တွေနဲ့ ဆိုင်တဲ့ character level symbol ဖိုင်ကို ထုတ်မယ်။  
input အတွက် ဆိုရင် --input ဆိုတဲ့ parameter ကို သုံးတယ်။  

In [147]:
!cat ../script/mk_symbol.py

"""
Generate input and output symbol tables.
Written by Ye Kyaw Thu, LU Lab., Myanmar.
Last update: 11 July 2025
Usage:
    python ../script/mk_symbol.py --input ./lexicon.txt > char.sym
    python ../script/mk_symbol.py --output ./lexicon.txt > word.sym
"""

#!/""usr/bin/""env python3
import sys
import argparse
from collections import defaultdict

def main():
    parser = argparse.ArgumentParser(description='Generate symbol table from FST files')
    parser.add_argument('--input', metavar='FILE', action='append', 
                       help='Input FST file(s) to read input symbols from')
    parser.add_argument('--output', metavar='FILE', action='append',
                       help='Output FST file(s) to read output symbols from')
    
    args = parser.parse_args()
    
    if not args.input and not args.output:
        parser.print_help()
        sys.exit(1)
    
    # Count the symbols to be printed
    toprint = defaultdict(int)
    
    special_symbols = ['<eps>', '<s>', '<unk>

In [148]:
!python ../script/mk_symbol.py --input ./lexicon.txt > char.sym

In [149]:
!wc char.sym

 271  542 1885 char.sym


In [150]:
!head -n 20 char.sym

<eps> 0
<s> 1
<unk> 2
<w> 3
% 4
& 5
' 6
* 7
+ 8
. 9
0 10
1 11
2 12
3 13
4 14
5 15
6 16
7 17
8 18
9 19


In [151]:
!tail char.sym

得 261
新 262
日 263
昭 264
時 265
本 266
甸 267
缅 268
語 269
﻿ 270


In [152]:
!shuf char.sym 2>/dev/null | head -n 20 

q 67
ャ 250
X 47
ṃ 225
æ 79
<s> 1
ဉ 162
प 113
ม 141
္ 204
ท 137
ေ 199
U 44
ศ 144
ส 145
Y 48
ူ 198
ु 121
ไ 152
ச 125


ဒီတခါတော့ output symbol ဖိုင် သို့မဟုတ် word level unit symbol ဖိုင်ကို ထုတ်မယ်။  
--output argument ကို သုံးရမယ်။  

In [153]:
!python ../script/mk_symbol.py --output ./lexicon.txt > word.sym

In [154]:
!wc word.sym

 24801  49602 756430 word.sym


In [155]:
!head -n 20 word.sym

<eps> 0
<s> 1
<unk> 2
<w> 3
% 4
' 5
'' 6
* 7
+ 8
. 9
.. 10
... 11
.... 12
0 13
0ရာသီလာ 14
1 15
1.5 16
1.57 17
10 18
103 19


In [156]:
!tail -n 20 word.sym

‌‌လေ့လာ 24781
‌‌‌ယောက္ခမ 24782
‌‌‌‌‌ဝေါဟာရ 24783
‘ 24784
’ 24785
“ 24786
” 24787
− 24788
おじいさん 24789
おじさん 24790
つ 24791
つなみ 24792
ビルマ人 24793
ミャンマー 24794
新加坡 24795
日本語 24796
昭南島 24797
昭和の時代に得た南の島 24798
缅甸 24799
﻿တင်ပို့ 24800


## Compilation



In [157]:
cd ..

/home/ye/exp/fst_decoder


In [185]:
!fstcompile --isymbols=./data/char.sym --osymbols=./data/word.sym ./data/lexicon.txt lexicon.fst

In [186]:
!fstprint ./lexicon.fst | head -n 20

0	1	4	4	1
0	2	6	5	1
0	3	6	6	1
0	5	7	7	1
0	6	8	8	1
0	7	9	9	1
0	8	9	10	1
0	10	9	11	1
0	13	9	12	1
0	17	10	13	1
0	18	10	14	1
0	25	11	15	1
0	26	11	16	1
0	29	11	17	1
0	33	11	18	1
0	35	11	19	1
0	38	11	20	1
0	43	11	21	1
0	47	11	22	1
0	49	11	23	1


In [187]:
!fstprint --isymbols=./data/char.sym --osymbols=./data/word.sym ./lexicon.fst | head -n 20

0	1	%	%	1
0	2	'	'	1
0	3	'	''	1
0	5	*	*	1
0	6	+	+	1
0	7	.	.	1
0	8	.	..	1
0	10	.	...	1
0	13	.	....	1
0	17	0	0	1
0	18	0	0ရာသီလာ	1
0	25	1	1	1
0	26	1	1.5	1
0	29	1	1.57	1
0	33	1	10	1
0	35	1	103	1
0	38	1	10pow	1
0	43	1	10th	1
0	47	1	12	1
0	49	1	123	1


## Optimization

In [188]:
!fstdeterminize lexicon.fst lexicon.det

In [189]:
!fstminimize lexicon.det lexicon.min

In [190]:
!fstarcsort lexicon.min lexicon.srt

Optimization မလုပ်ခင်က .fst ဖိုင်နဲ့ .srt ဖိုင်ရဲ့ filesize ကို နှိုင်းယှဉ်ကြည့်ရင် ကွာတာကို မြင်ရပါလိမ့်မယ်။  

In [191]:
!ls -lh lexicon.fst lexicon.srt

-rw-rw-r-- 1 ye ye 6.0M Jul 13 01:35 lexicon.fst
-rw-rw-r-- 1 ye ye 1.3M Jul 13 01:35 lexicon.srt


srt ဖိုင်ကို text ဖိုင်အဖြစ်ပြောင်းလို့ ရတယ်။  

In [165]:
!fstprint --isymbols=./data/char.sym --osymbols=./data/word.sym lexicon.srt > lexicon.srt.txt

In [166]:
!head -n 30 lexicon.srt.txt

0	0	<s>	<s>
0	1	%	%	1
0	2	'	<eps>	1
0	1	*	*	1
0	1	+	+	1
0	3	.	<eps>	1
0	4	0	<eps>	1
0	5	1	<eps>	1
0	6	2	<eps>	1
0	7	3	<eps>	1
0	8	4	<eps>	1
0	1	5	5	1
0	9	6	<eps>	1
0	10	7	<eps>	1
0	11	8	<eps>	1
0	12	9	<eps>	1
0	1	:	:	1
0	1	;	;	1
0	1	=	=	1
0	13	@	<eps>	1
0	14	A	<eps>	1
0	15	B	<eps>	1
0	16	C	<eps>	1
0	17	D	<eps>	1
0	18	E	<eps>	1
0	19	F	<eps>	1
0	20	G	<eps>	1
0	21	H	<eps>	1
0	22	I	<eps>	1
0	23	J	<eps>	1


In [167]:
!tail -n 20 lexicon.srt.txt

34137	21800	က	<eps>
34138	6680	ျ	<eps>
34139	34142	စ	<eps>
34140	34143	ဟ	<eps>
34141	34144	ိ	<eps>
34142	34145	ိ	<eps>
34143	34146	ာ	<eps>
34144	23117	က	<eps>
34145	34147	ု	<eps>
34146	34148	ဇ	<eps>
34147	34149	င	<eps>
34148	34150	ေ	<eps>
34149	34151	်	<eps>
34150	34152	ယ	<eps>
34151	17684	က	<eps>
34152	34153	သ	<eps>
34153	34154	င	<eps>
34154	34155	်	<eps>
34155	34156	္	<eps>
34156	4937	ခ	<eps>


## Preparation for Decoding

ဒီနေရာမှာ word boundary ကိစ္စကို ဘယ်လိုသတ်မှတ်ကြမလဲ စဉ်းစားရလိမ့်မယ်။  
<w> ဆိုတဲ့ သင်္ကေတကိုပဲသုံးပြီး စာလုံးတိုင်းရဲ့ နောက်မှာ လိုက်ဖြည့်ရင်လည်း ရပေမဲ့ lexicon or dictionry ဖိုင်ကလည်း ရှိပြီးသားမို့ <w> ကို <eps> နဲ့ အစားထိုးလုပ်သွားမယ်။   

In [112]:
!echo "3 0" > map.txt

In [113]:
!cat map.txt

3 0


In [168]:
!fstrelabel --relabel_ipairs=map.txt lexicon.srt lexicon_final.fst

ပြန်စစ်ကြည့်ရအောင်  

In [169]:
!fstprint --isymbols=./data/char.sym --osymbols=./data/word.sym lexicon_final.fst > lexicon_final.fst.txt

In [170]:
!head -n 6000 ./lexicon_final.fst.txt | tail -n 50 

1208	4791	ု	<eps>
1209	0	<eps>	အီ
1209	4795	က	<eps>
1209	4804	ဂ	အီဂျစ်
1209	4805	စ	အီစီဂျီ
1209	771	ဆ	အီဆာ
1209	4796	တ	<eps>
1209	4797	ဒ	<eps>
1209	4798	ဖ	<eps>
1209	4806	ဗ	အီဗော်လျူရှင်
1209	4799	မ	<eps>
1209	4800	ရ	<eps>
1209	4801	လ	<eps>
1209	837	ဝ	အီဝီ
1209	4802	သ	<eps>
1209	4803	း	<eps>
1210	4807	တ	<eps>
1210	4808	န	<eps>
1210	4809	ပ	<eps>
1210	4810	ံ	<eps>
1211	0	<eps>	အူ
1211	4811	က	<eps>
1211	4812	တ	<eps>
1211	4815	ပ	အူပျော်မွေ့
1211	1	မ	အူမ
1211	4816	ယ	အူယောင်ငန်းဖျား
1211	4813	ရ	<eps>
1211	4817	လ	အူလ်ဆန်
1211	4814	အ	<eps>
1211	4818	း	အူးလ်ဆန်
1212	0	<eps>	အေ
1212	4824	ဂ	အေဂျင်စီ
1212	837	ဒ	အေဒီ
1212	837	ပ	အေပီ
1212	4819	ဗ	<eps>
1212	4822	ဘ	<eps>
1212	4531	ယ	အေယူ
1212	4820	ရ	<eps>
1212	771	လ	အေလာ
1212	4825	အ	အေအိုင်ဒီအက်စ်
1212	4821	ာ	<eps>
1212	4826	့	အေ့မက်ထရစ်
1212	4823	း	<eps>
1213	0	<eps>	အဲ
1213	4833	က	အဲကွန်း
1213	4827	ဒ	<eps>
1213	4834	ဖ	အဲဖရန့်စ်
1213	4828	ဗ	<eps>
1213	4829	ယ	<eps>
1213	4830	ရ	<eps>


## Configuration File
fst_decoder.py ကို မသုံးခင်မှာ configuration ဖိုင်ကို ကြိုတင်ပြင်ဆင်ထားဖို့ လိုအပ်တယ်။
yaml format ကို သုံးထားတယ်။  

In [171]:
!cat config.yaml

input_format: text
output_format: text
nbest: 1
input_symbols: ./data/char.sym
output_symbols: ./data/word.sym
unknown_symbol: <unk>
terminal_symbol: </s>
models:
  - type: plain
    file: ./lexicon_final.fst
    input_symbols: ./data/char.sym
    output_symbols: ./data/word.sym
    weights: [1.0]


## Character Segmentation
Input လုပ်တဲ့အခါမှာ character တစ်လုံးချင်းစီဖြတ်ထားတဲ့ ပုံစံနဲ့ input လုပ်မှာမို့ test data ဖိုင်တွေကို character segmentation လုပ်ဖို့ လိုအပ်တယ်။  

In [172]:
!python ./script/char_break.py < ./data/ctest.word > ./data/ctest.char

In [173]:
!head ./data/ctest.char

ပ င ် လ ယ ် န ဲ ့ မ ျ က ် န ှ ာ ခ ျ င ် း ဆ ိ ု င ် တ စ ် ခ န ် း
သ ု ံ း န ှ စ ် ပ ျ မ ် း မ ျ ှ တ စ ် ဦ း ခ ျ င ် း တ စ ် န ှ စ ် စ ု စ ု ပ ေ ါ င ် း ဝ င ် င ွ ေ အ မ ေ ရ ိ က န ် ဒ ေ ါ ် လ ာ ၇ ၅ ဝ ထ က ် လ ျ ေ ာ ့ န ည ် း ရ မ ည ် ဖ ြ စ ် သ ည ်
ဘ ာ သ ာ စ က ာ း မ ှ ာ ပ ါ ရ မ ီ ပ ါ တ ယ ်
ဟ င ့ ် အ င ် း မ ပ ျ က ် ပ ါ ဘ ူ း
ပ ြ တ င ် း ပ ေ ါ က ် အ ပ ြ င ် ဘ က ် က ိ ု က ြ ည ့ ် လ ိ ု က ် တ ဲ ့ အ ခ ါ န ှ င ် း တ ွ ေ က ျ န ေ ခ ဲ ့ တ ယ ်
သ ူ အ ိ မ ် မ ှ ာ မ ရ ှ ိ ဘ ူ း
မ ြ ေ ာ က ် အ မ ေ ရ ိ က တ ိ ု က ် တ ွ င ် တ ွ ေ ့ ရ သ ေ ာ ဝ က ် ဝ ံ မ ျ ိ ု း မ ျ ာ း အ န က ် ဝ က ် ဝ ံ မ ည ် း သ ည ် ဝ က ် ဝ ံ မ ျ ိ ု း တ ွ င ် အ တ ွ ေ ့ ရ အ မ ျ ာ း ဆ ု ံ း အ မ ျ ိ ု း ဖ ြ စ ် သ ည ်
က ျ ွ န ် တ ေ ာ ် ခ င ် ဗ ျ ာ း ရ ဲ ့ က ာ း မ ေ ာ င ် း လ ိ ု င ် စ င ် အ မ ည ် လ ိ ပ ် စ ာ န ဲ ့ ဖ ု န ် း န ံ ပ ါ တ ် တ ိ ု ့ က ိ ု လ ည ် း တ ခ ျ က ် က ြ ည ် ့ ခ ျ င ် ပ ါ တ ယ ်
အ ခ န ် း ရ ေ ာ က ် သ ေ ာ ်
န ေ ာ က ် အ ပ တ ် က ျ ွ န ် တ ေ ာ ် တ ိ ု ့ S u z h o u က ိ ု သ ွ ာ း က ြ မ လ ိ ု ့


**closed test ဒေတာကို character segmentation ဖြတ်တာ ပြီးပြီမို့ open test ဒေတာကိုလည်း character segmentation လုပ်ထားမယ်။**  

In [174]:
!python ./script/char_break.py < ./data/otest.nopipe.word > ./data/otest.char 

In [175]:
!head ./data/otest.char

ဆ ယ ် ရ ာ ခ ိ ု င ် န ှ ု န ် း ဈ ေ း လ ျ ှ ေ ာ ့ ပ ေ း ရ င ် ဝ ယ ် မ ယ ် ။
ယ ခ ု လ ၏ အ ထ ိ မ ် း အ မ ှ တ ် ပ န ် း မ ှ ာ မ ြ တ ် လ ေ း ပ န ် း P o m e a c o c c i n e a ဖ ြ စ ် သ ည ် ။
က ရ င ် ဗ မ ာ အ ဓ ိ က ရ ု ဏ ် း သ ည ် သ ူ ့ အ ာ း မ ျ ာ း စ ွ ာ ဒ ေ ါ မ န ဿ ဖ ြ စ ် စ ေ ပ ါ သ ည ် ။
က ျ ေ ာ က ု န ် း ၏ အ လ ယ ် သ ည ် ဝ က ် ဝ ံ ၏ တ စ ် က ိ ု ယ ် လ ု ံ း တ ွ င ် အ မ ြ င ် ့ ဆ ု ံ း အ စ ိ တ ် အ ပ ိ ု င ် း ဖ ြ စ ် သ ည ် ။
သ ိ ု ့ သ ေ ာ ် ၂ ၀ ၀ ၄ ခ ု န ှ စ ် မ ှ စ တ င ် ၍ အ စ ိ ု း ရ ၏ န ိ ု င ် င ံ ရ ေ း လ ှ ု ပ ် ရ ှ ာ း မ ှ ု မ ျ ာ း က ိ ု က န ် ့ သ တ ် သ ေ ာ စ စ ် ဆ င ် ရ ေ း တ စ ် ရ ပ ် အ န ေ ဖ ြ င ် ့ ၎ င ် း ရ ု ံ း ခ ွ ဲ မ ျ ာ း က ိ ု ပ ြ န ် လ ည ် ပ ိ တ ် စ ေ ခ ြ င ် း သ ိ ု ့ မ ဟ ု တ ် ပ ြ င ် း ထ န ် သ ေ ာ ပ ိ တ ် ပ င ် တ ာ း ဆ ီ း မ ှ ု မ ျ ာ း က ိ ု ပ ြ ု လ ု ပ ် ခ ဲ ့ သ ည ် ။
အ ပ ိ ု အ လ ျ ှ ံ မ ျ ာ း က ိ ု န ိ ု င ် င ံ တ ေ ာ ် အ ဖ ွ ဲ ့ အ စ ည ် း မ ျ ာ း သ ိ ု ့ တ ိ ု က ် ရ ိ ု က ် ရ ေ ာ င ် း ခ ျ သ ည ် ။
ထ ိ ု ထ က ် တ ိ က ျ စ ွ ာ ဖ ေ ာ ် ပ ြ ဦ း မ ည ် ။
ဆ ရ ာ က ြ ီ း ၏ ဇ 

## Decoding
လက်ရှိ လုပ်ပြတဲ့ ဒီမိုမှာတော့ decoding လုပ်တယ်ဆိုတာက word segmentation လုပ်တာပေါ့။  
fst_decoder.py code ကို လေ့လာချင်သူတွေအတွက် cat နဲ့ ရိုက်ထုတ်ပြမယ်။  

In [176]:
!cat fst_decoder.py 

"""
FST Decoder
Written by Ye Kyaw Thu
Last Update: 11 July 2025

Usage:
    python ./fst_decoder.py ./config.yaml --show-id < ./closed_test.txt > closed_test.hyp
    python ./fst_decoder.py ./config.yaml --show-id < ./open_test.txt > open_test.hyp

Reference: https://www.phontron.com/kyfd/
"""

import os
import sys
import yaml
import argparse
import time
from typing import List, Dict, Any
import pywrapfst as fst
import pynini
import tempfile

class ConfigError(Exception):
    pass

class DecoderError(Exception):
    pass

class FSTModel:
    def __init__(self, config, symbol_tables=None):
        """Initialize the FST model with configuration"""
        self.config = config
        self.symbol_tables = symbol_tables or {}
        self.fst = None
        
    def load(self):
        """Load the FST with explicit symbol tables"""
        try:
            fst_file = self.config.get('file')
            if not fst_file:
                raise ConfigError("No FST file specified in config")
 

## Decoding with Closed Test Data

In [177]:
!time python ./fst_decoder.py ./config.yaml < ./data/ctest.char > ./closed.hyp


Initializing model with config: {'type': 'plain', 'file': './lexicon_final.fst', 'input_symbols': './data/char.sym', 'output_symbols': './data/word.sym', 'weights': [1.0]}
Model object created, attempting to load...
Attempting to load FST from: ./lexicon_final.fst
Successfully loaded FST with 34157 states
Input symbols: yes
Output symbols: yes
Model loaded successfully. Start state: 0
Num states: 34157
Input symbols: <const Fst SymbolTableView './data/char.sym' at 0x7c90abf330c0>
Output symbols: <const Fst SymbolTableView './data/word.sym' at 0x7c90abf330c0>
-- Starting FST Decoder --
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with 

In [178]:
!wc closed.hyp

  1000  10550 161125 closed.hyp


In [179]:
!head closed.hyp

ပင်လယ် နဲ့ မျက်နှာချင်းဆိုင် တစ်ခန်း
သုံး နှစ် ပျမ်းမျှ တစ် ဦး ချင်း တစ် နှစ် စုစုပေါင်း ဝင်ငွေ အမေရိကန်ဒေါ်လာ ၇၅ဝ ထက် လျော့နည်း ရ မည် ဖြစ် သည်
ဘာသာစကား မှာ ပါရမီ ပါတယ်
ဟင့်အင်း မ ပျက် ပါဘူး
ပြတင်းပေါက် အပြင်ဘက် ကို ကြည့် လိုက် တဲ့ အခါ နှင်း တွေ ကျ နေ ခဲ့ တယ်
သူ အိမ် မှာ မရှိ ဘူး
မြောက်အမေရိက တိုက် တွင် တွေ့ ရ သော ဝက်ဝံ မျိုး များ အနက် ဝက်ဝံမည်း သည် ဝက်ဝံ မျိုး တွင် အတွေ့ ရ အများဆုံး အမျိုး ဖြစ် သည်
ကျွန်တော် ခင်ဗျားရဲ့ ကားမောင်းလိုင်စင် အမည် လိပ်စာ နဲ့ ဖုန်းနံပါတ် တို့ ကို လည်း တချက် ကြည့် ချင်ပါတယ်
အခန်း ရောက် သော်
နောက် အပတ် ကျွန်တော် တို့ Suzhou ကို သွား ကြ မလို့


## Evaluation for Closed-Test Data

ဒီ evaluation script က ဆရာ NICT, Kyoto က Multilingual Machine Translation Lab. မှာ အလုပ်လုပ်ကတည်းက သုံးခဲ့တဲ့ python code ပါ။ အဲဒါကြောင့် run မယ် ဆိုရင် python2.7 နဲ့မှ အဆင်ပြေပါတယ်။ ရေးထားတဲ့ Author က Yue Zhang, University of Oxford ပါ။   
code ကို လေ့လာချင်သူတွေအတွက် cat နဲ့ ရိုက်ထုတ်ပြမယ်။  

In [180]:
!cat ./script/evaluate.py

#****************************************************************
#
# evaluate.py - the evaluation program.
#
# Author: Yue Zhang
#
# Computing lab, University of Oxford. 2006.11
#
#****************************************************************

#================================================================
#
# Import modules.
#
#================================================================

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "libs")))
import getopt

#----------------------------------------------------------------
#
# addTuples - add two tuples element by element.
#
# Inputs:  tuple1 - operand1
#          tuple2 - operand2
#
# Returns: tuple
#
#----------------------------------------------------------------

def addTuples(tuple1, tuple2):
   return tuple([tuple1[i]+tuple2[i] for i in xrange(len(tuple1))])

#----------------------------------------------------------------
#
# addListToList - add the second list to

In [181]:
!python2.7 ./script/evaluate.py ./closed.hyp ./data/ctest.word

Tag precision: 0.916303317536


## Decoding with Open-Test Data
ဒီတခါတော့ training လုပ်စဉ်က ဖယ်ထားတဲ့ (သို့) training data ထဲမှာ မပါတဲ့ စာကြောင်းတွေကို word segmentation လုပ်ခိုင်းကြည့်မယ်။   
မော်ဒယ်တွေအတွက်က အဲဒီလို open-test data က စာလုံးမှန်အောင် ဖြတ်ပေးနိုင်ဖို့ ခက်တယ်။  
အဲဒါကြောင့် experiment အနေနဲ့ လုပ်တဲ့အခါမှာ open-test လုပ်ဖို့လိုအပ်တယ်။  

In [183]:
!time python ./fst_decoder.py ./config.yaml < ./data/otest.char > otest.hyp


Initializing model with config: {'type': 'plain', 'file': './lexicon_final.fst', 'input_symbols': './data/char.sym', 'output_symbols': './data/word.sym', 'weights': [1.0]}
Model object created, attempting to load...
Attempting to load FST from: ./lexicon_final.fst
Successfully loaded FST with 34157 states
Input symbols: yes
Output symbols: yes
Model loaded successfully. Start state: 0
Num states: 34157
Input symbols: <const Fst SymbolTableView './data/char.sym' at 0x77dfcb480f90>
Output symbols: <const Fst SymbolTableView './data/word.sym' at 0x77dfcb480f90>
-- Starting FST Decoder --
Common symbols: 271
Composing with model: ./lexicon_final.fst
Composition failed - possible symbol mismatch
Input FST symbols: <const Fst SymbolTableView './data/char.sym' at 0x77dfc9f7ba50>
Model FST symbols: <const Fst SymbolTableView './data/char.sym' at 0x77dfc9f7ba50>
Common symbols: 271
Composing with model: ./lexicon_final.fst
Composition failed - possible symbol mismatch
Input FST symbols: <const

In [184]:
!wc ./open_test.hyp

0 0 0 ./open_test.hyp


## Debugging

Error ဖြစ်ရတဲ့ အကြောင်းအရင်းက ပုဒ်ထီး၊ ပုဒ်မ က version1 ရဲ့ open test data မှာ ပါနေလို့လို့ ယူဆ။  

In [192]:
!pwd

/home/ye/exp/fst_decoder


In [193]:
!python ./script/rm_myanmar_punct.py --help

usage: rm_myanmar_punct.py [-h] [--input INPUT] [--output OUTPUT]

Remove Myanmar punctuation characters (၊ and ။) from text.

options:
  -h, --help       show this help message and exit
  --input INPUT    Input file path (default: stdin) (default: None)
  --output OUTPUT  Output file path (default: stdout) (default: None)


In [194]:
!python ./script/rm_myanmar_punct.py --input ./data/otest.nopipe.word --output ./data/otest.nopipe.word.no_punct

In [195]:
!head ./data/otest.nopipe.word

ဆယ် ရာခိုင်နှုန်း ဈေး လျှော့ ပေး ရင် ဝယ် မယ် ။
ယခု လ ၏ အထိမ်းအမှတ် ပန်း မှာ မြတ်လေးပန်း Pomeacoccinea ဖြစ် သည် ။
ကရင် ဗမာ အဓိကရုဏ်း သည် သူ့ အား များ စွာ ဒေါမနဿ ဖြစ် စေ ပါ သည် ။
ကျောကုန်း ၏ အလယ် သည် ဝက်ဝံ ၏ တစ်ကိုယ်လုံး တွင် အမြင့်ဆုံး အစိတ်အပိုင်း ဖြစ် သည် ။
သို့သော် ၂၀၀၄ ခုနှစ် မှ စတင် ၍ အစိုးရ ၏ နိုင်ငံ ရေး လှုပ်ရှား မှု များ ကို ကန့်သတ် သော စစ်ဆင် ရေး တစ် ရပ် အနေဖြင့် ၎င်း ရုံးခွဲ များ ကို ပြန်လည် ပိတ် စေ ခြင်း သို့မဟုတ် ပြင်းထန် သော ပိတ်ပင် တားဆီး မှု များ ကို ပြုလုပ် ခဲ့ သည် ။
အပိုအလျှံ များ ကို နိုင်ငံတော် အဖွဲ့အစည်း များ သို့ တိုက်ရိုက် ရောင်းချ သည် ။
ထို ထက် တိကျ စွာ ဖော်ပြ ဦး မည် ။
ဆရာကြီး ၏ ဇနီး ဖြစ် သူ ဒေါ်စိုး မ သေ ခင် ရွှေဥဒေါင်း စာမူ ကိစ္စ နှင့် ပတ်သက် ၍ သေတမ်းစာ ရေး ခဲ့ သည် ။
မြစ်ကြီးနား မှ ၁၁၈ မိုင် အကွာ တွင် တည်ရှိ သည် ။
ကုန်းလိပ် နှင့် ပင်လယ်လိပ် တို့ သည် ကိလိုနီယာ မျိုးစဉ် တွင် ပါဝင် လေ သည် ။


In [196]:
!head ./data/otest.nopipe.word.no_punct

ဆယ် ရာခိုင်နှုန်း ဈေး လျှော့ ပေး ရင် ဝယ် မယ် 
ယခု လ ၏ အထိမ်းအမှတ် ပန်း မှာ မြတ်လေးပန်း Pomeacoccinea ဖြစ် သည် 
ကရင် ဗမာ အဓိကရုဏ်း သည် သူ့ အား များ စွာ ဒေါမနဿ ဖြစ် စေ ပါ သည် 
ကျောကုန်း ၏ အလယ် သည် ဝက်ဝံ ၏ တစ်ကိုယ်လုံး တွင် အမြင့်ဆုံး အစိတ်အပိုင်း ဖြစ် သည် 
သို့သော် ၂၀၀၄ ခုနှစ် မှ စတင် ၍ အစိုးရ ၏ နိုင်ငံ ရေး လှုပ်ရှား မှု များ ကို ကန့်သတ် သော စစ်ဆင် ရေး တစ် ရပ် အနေဖြင့် ၎င်း ရုံးခွဲ များ ကို ပြန်လည် ပိတ် စေ ခြင်း သို့မဟုတ် ပြင်းထန် သော ပိတ်ပင် တားဆီး မှု များ ကို ပြုလုပ် ခဲ့ သည် 
အပိုအလျှံ များ ကို နိုင်ငံတော် အဖွဲ့အစည်း များ သို့ တိုက်ရိုက် ရောင်းချ သည် 
ထို ထက် တိကျ စွာ ဖော်ပြ ဦး မည် 
ဆရာကြီး ၏ ဇနီး ဖြစ် သူ ဒေါ်စိုး မ သေ ခင် ရွှေဥဒေါင်း စာမူ ကိစ္စ နှင့် ပတ်သက် ၍ သေတမ်းစာ ရေး ခဲ့ သည် 
မြစ်ကြီးနား မှ ၁၁၈ မိုင် အကွာ တွင် တည်ရှိ သည် 
ကုန်းလိပ် နှင့် ပင်လယ်လိပ် တို့ သည် ကိလိုနီယာ မျိုးစဉ် တွင် ပါဝင် လေ သည် 


In [197]:
!python ./script/char_break.py --input ./data/otest.nopipe.word.no_punct --output ./data/otest.nopipe.word.no_punct.char

In [198]:
!head ./data/otest.nopipe.word.no_punct.char

ဆ ယ ် ရ ာ ခ ိ ု င ် န ှ ု န ် း ဈ ေ း လ ျ ှ ေ ာ ့ ပ ေ း ရ င ် ဝ ယ ် မ ယ ်
ယ ခ ု လ ၏ အ ထ ိ မ ် း အ မ ှ တ ် ပ န ် း မ ှ ာ မ ြ တ ် လ ေ း ပ န ် း P o m e a c o c c i n e a ဖ ြ စ ် သ ည ်
က ရ င ် ဗ မ ာ အ ဓ ိ က ရ ု ဏ ် း သ ည ် သ ူ ့ အ ာ း မ ျ ာ း စ ွ ာ ဒ ေ ါ မ န ဿ ဖ ြ စ ် စ ေ ပ ါ သ ည ်
က ျ ေ ာ က ု န ် း ၏ အ လ ယ ် သ ည ် ဝ က ် ဝ ံ ၏ တ စ ် က ိ ု ယ ် လ ု ံ း တ ွ င ် အ မ ြ င ် ့ ဆ ု ံ း အ စ ိ တ ် အ ပ ိ ု င ် း ဖ ြ စ ် သ ည ်
သ ိ ု ့ သ ေ ာ ် ၂ ၀ ၀ ၄ ခ ု န ှ စ ် မ ှ စ တ င ် ၍ အ စ ိ ု း ရ ၏ န ိ ု င ် င ံ ရ ေ း လ ှ ု ပ ် ရ ှ ာ း မ ှ ု မ ျ ာ း က ိ ု က န ် ့ သ တ ် သ ေ ာ စ စ ် ဆ င ် ရ ေ း တ စ ် ရ ပ ် အ န ေ ဖ ြ င ် ့ ၎ င ် း ရ ု ံ း ခ ွ ဲ မ ျ ာ း က ိ ု ပ ြ န ် လ ည ် ပ ိ တ ် စ ေ ခ ြ င ် း သ ိ ု ့ မ ဟ ု တ ် ပ ြ င ် း ထ န ် သ ေ ာ ပ ိ တ ် ပ င ် တ ာ း ဆ ီ း မ ှ ု မ ျ ာ း က ိ ု ပ ြ ု လ ု ပ ် ခ ဲ ့ သ ည ်
အ ပ ိ ု အ လ ျ ှ ံ မ ျ ာ း က ိ ု န ိ ု င ် င ံ တ ေ ာ ် အ ဖ ွ ဲ ့ အ စ ည ် း မ ျ ာ း သ ိ ု ့ တ ိ ု က ် ရ ိ ု က ် ရ ေ ာ င ် း ခ ျ သ ည ်
ထ ိ ု ထ က ် တ ိ က ျ စ ွ ာ ဖ ေ ာ ် ပ ြ ဦ း မ ည ်
ဆ ရ ာ က ြ ီ း ၏ ဇ န ီ း ဖ ြ စ ် 

## Decoding with Open-Test Data Again

In [199]:
!time python ./fst_decoder.py ./config.yaml < ./data/otest.nopipe.word.no_punct.char > otest.hyp2


Initializing model with config: {'type': 'plain', 'file': './lexicon_final.fst', 'input_symbols': './data/char.sym', 'output_symbols': './data/word.sym', 'weights': [1.0]}
Model object created, attempting to load...
Attempting to load FST from: ./lexicon_final.fst
Successfully loaded FST with 34157 states
Input symbols: yes
Output symbols: yes
Model loaded successfully. Start state: 0
Num states: 34157
Input symbols: <const Fst SymbolTableView './data/char.sym' at 0x761010b5b630>
Output symbols: <const Fst SymbolTableView './data/word.sym' at 0x761010b5b630>
-- Starting FST Decoder --
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with 

In [200]:
!wc ./otest.hyp2

   983  19043 302098 ./otest.hyp2


In [201]:
!head ./otest.hyp2

ဆယ် ရာခိုင်နှုန်း ဈေးလျှော့ပေး ရင် ဝယ် မယ်
ယခု လ ၏ အထိမ်းအမှတ် ပန်း မှာ မြတ်လေးပန်း Pomeacoccinea ဖြစ် သည်
ကရင် ဗမာ အဓိကရုဏ်း သည် သူ့ အား များစွာ ဒေါမနဿ ဖြစ်စေ ပါ သည်
ကျောကုန်း ၏ အလယ် သည် ဝက်ဝံ ၏ တစ်ကိုယ်လုံး တွင် အမြင့်ဆုံး အစိတ်အပိုင်း ဖြစ် သည်
သို့သော် ၂၀၀၄ ခုနှစ် မှ စတင် ၍ အစိုးရ ၏ နိုင်ငံရေး လှုပ်ရှားမှု များ ကို ကန့်သတ် သော စစ်ဆင် ရေး တစ် ရပ် အနေဖြင့် ၎င်း ရုံးခွဲ များ ကို ပြန်လည် ပိတ် စေ ခြင်း သို့မဟုတ် ပြင်းထန် သော ပိတ်ပင် တားဆီး မှု များ ကို ပြုလုပ် ခဲ့ သည်
အပိုအလျှံ များ ကို နိုင်ငံတော် အဖွဲ့အစည်း များ သို့ တိုက်ရိုက် ရောင်းချ သည်
ထို ထက် တိကျ စွာ ဖော်ပြ ဦး မည်
ဆရာကြီး ၏ ဇနီး ဖြစ် သူ ဒေါ်စိုး မ သေ ခင် ရွှေဥဒေါင်း စာမူ ကိစ္စ နှင့် ပတ်သက် ၍ သေတမ်းစာ ရေး ခဲ့ သည်
မြစ်ကြီးနား မှ ၁၁၈ မိုင် အကွာ တွင် တည်ရှိ သည်
ကုန်းလိပ် နှင့် ပင်လယ်လိပ် တို့ သည် ကိလိုနီယာ မျိုးစဉ် တွင် ပါဝင် လေ သည်


In [202]:
!python2.7 ./script/evaluate.py ./otest.hyp2 ./data/otest.nopipe.word.no_punct

Tag precision: 0.0249960615449


စာကြောင်းရေ အရေအတွက် မတူတာကြောင့် evaluation score က တအားနည်းနေတယ်။  
Evaluation score တွက်တာက မှားနေတယ်။  
အဲဒါကြောင့် reference နဲ့ hypothesis ဖိုင်နှစ်ဖိုင်ရဲ့ စာကြောင်းရေ အရေအတွက်မတူတဲ့အခါမှာတွက်ပေးနိုင်တဲ့ evaluate_setmentation.py code ကို ရေးခဲ့တယ်။  

In [203]:
!python ./script/evaluate_segmentation.py --help

usage: evaluate_segmentation.py [-h] --ref REF --hyp HYP [--top-k TOP_K]

Boundary-based evaluation for word segmentation (Precision, Recall, F1 +
Token-level Top-K Errors)

options:
  -h, --help     show this help message and exit
  --ref REF      Path to reference (gold) word segmented file
  --hyp HYP      Path to hypothesis file with line ID and segmented output
  --top-k TOP_K  Top-K most frequent token-level segmentation errors to report
                 (default: 10)


Help နဲ့ ကြည့်ရင် မြင်ရတဲ့အတိုင်းပဲ HYP ဖိုင်မှာ ID လိုအပ်တယ်။  
## Decoding Open-Test Data with ID

In [204]:
!time python ./fst_decoder.py ./config.yaml --show-id < ./data/otest.nopipe.word.no_punct.char > otest.hyp3


Initializing model with config: {'type': 'plain', 'file': './lexicon_final.fst', 'input_symbols': './data/char.sym', 'output_symbols': './data/word.sym', 'weights': [1.0]}
Model object created, attempting to load...
Attempting to load FST from: ./lexicon_final.fst
Successfully loaded FST with 34157 states
Input symbols: yes
Output symbols: yes
Model loaded successfully. Start state: 0
Num states: 34157
Input symbols: <const Fst SymbolTableView './data/char.sym' at 0x7c77b2f77720>
Output symbols: <const Fst SymbolTableView './data/word.sym' at 0x7c77b2f77720>
-- Starting FST Decoder --
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with model: ./lexicon_final.fst
Common symbols: 271
Composing with 

In [205]:
!wc ./otest.hyp3

   983  19043 307888 ./otest.hyp3


In [206]:
!head ./otest.hyp3

0|||ဆယ် ရာခိုင်နှုန်း ဈေးလျှော့ပေး ရင် ဝယ် မယ်
1|||ယခု လ ၏ အထိမ်းအမှတ် ပန်း မှာ မြတ်လေးပန်း Pomeacoccinea ဖြစ် သည်
2|||ကရင် ဗမာ အဓိကရုဏ်း သည် သူ့ အား များစွာ ဒေါမနဿ ဖြစ်စေ ပါ သည်
3|||ကျောကုန်း ၏ အလယ် သည် ဝက်ဝံ ၏ တစ်ကိုယ်လုံး တွင် အမြင့်ဆုံး အစိတ်အပိုင်း ဖြစ် သည်
4|||သို့သော် ၂၀၀၄ ခုနှစ် မှ စတင် ၍ အစိုးရ ၏ နိုင်ငံရေး လှုပ်ရှားမှု များ ကို ကန့်သတ် သော စစ်ဆင် ရေး တစ် ရပ် အနေဖြင့် ၎င်း ရုံးခွဲ များ ကို ပြန်လည် ပိတ် စေ ခြင်း သို့မဟုတ် ပြင်းထန် သော ပိတ်ပင် တားဆီး မှု များ ကို ပြုလုပ် ခဲ့ သည်
5|||အပိုအလျှံ များ ကို နိုင်ငံတော် အဖွဲ့အစည်း များ သို့ တိုက်ရိုက် ရောင်းချ သည်
6|||ထို ထက် တိကျ စွာ ဖော်ပြ ဦး မည်
7|||ဆရာကြီး ၏ ဇနီး ဖြစ် သူ ဒေါ်စိုး မ သေ ခင် ရွှေဥဒေါင်း စာမူ ကိစ္စ နှင့် ပတ်သက် ၍ သေတမ်းစာ ရေး ခဲ့ သည်
8|||မြစ်ကြီးနား မှ ၁၁၈ မိုင် အကွာ တွင် တည်ရှိ သည်
9|||ကုန်းလိပ် နှင့် ပင်လယ်လိပ် တို့ သည် ကိလိုနီယာ မျိုးစဉ် တွင် ပါဝင် လေ သည်


In [207]:
!tail ./otest.hyp3

990|||သို့သော် အာရပ် တို့ သည် ခရစ် ပေါ် ပြီး မှ မြင်း ကို အသုံးပြု ကြ ကြောင်း သိ ရ သည်
991|||ဓား အောက်ကျ သွား ပြီ
992|||ပတ်စပို့ ပြ ပါ
993|||ယင်း ဝေဖန် မှု မှာ ဤ လှူဒါန်း ငွေများ မှာ ငွေ အရင်းအနှီး အများအပြား ကို ရင်းနှီးမြှုပ်နှံ ခြင်း ဖြစ် ပြီး ထို ရင်းနှီးမြှုပ်နှံ မှု မှာ အမြင့်ဆုံး သော အမြတ် ကို ပြန်လည် ရရှိ ခြင်း မရှိ သေး ဘဲ ဖြစ် နေ သည် ဟူ၍ ဖြစ် သည်
994|||ကမ္ဘာတန်း ဆာရာဇမန္တဏ်ယောဂယာတြာ နှစ် ကျမ်း တွင် ဘောဇကဋတိုင်းသားသေတဗျ အမျိုး ဘဂုရ ရသေ့ မယား သုဇိတာ သား သောကြာ ဖြစ် ကြောင်း ကို ဆို သည်
995|||ယင်းနောက် ခရီး အတော် ကွာ မှ အင်္ဂလိပ် စကား နှင့် ဟင်ဒူစတန်နီ စကား တို့ လိုက် ပေ သည်
996|||ထို့ပြင် ဗိုလ် သင်တန်း နှင့် စစ် တက္ကသိုလ် ကြောင့် လည်း မေမြို့ သည် ယခု အခါ တွင် ထင်ရှား သည်
997|||ဤ စာအုပ် တွင် လည်း ထို ကဲ့သို့ တင်ပြ ခြင်း မျိုး အတိုင်းအတာ တစ် ခု အထိ ပါဝင် ခဲ့ ပါ လျှင် ထို အတိုင်းအတာ အထိ လွဲမှား သော အမြင်ရှိ ခဲ့ သည် ဟု ဆို နိုင် ပါ သည်
998|||ထို သို့ မ ထင် မိ ခဲ့ သဖြင့် သူ့ အား အပြစ်မတင် ထိုက် ပေ
999|||ထို မှတစ်ပါး ငှက် တို့ သည် ဟင်းလျာ အတွက် လည်း အသုံးဝင် သည်


In [209]:
!python ./script/evaluate_segmentation.py --ref ./data/otest.nopipe.word.no_punct --hyp ./otest.hyp3 --top 20


=== Word Segmentation Evaluation ===
Total Ref lines: 1000
Total Hyp lines: 983

Precision: 0.9949
Recall:    0.9205
F1 Score:  0.9563

Top-20 Frequent Token-level Segmentation Errors:
[1] Count: 40
  REF token:     မြန်မာ
  Predicted as:  မြန်မာနိုင်ငံ

[2] Count: 40
  REF token:     နိုင်ငံ
  Predicted as:  မြန်မာနိုင်ငံ

[3] Count: 29
  REF token:     ပါ
  Predicted as:  ပါတယ်

[4] Count: 29
  REF token:     တယ်
  Predicted as:  ပါတယ်

[5] Count: 26
  REF token:     မ
  Predicted as:  မရှိ

[6] Count: 26
  REF token:     ရှိ
  Predicted as:  မရှိ

[7] Count: 23
  REF token:     များ
  Predicted as:  များစွာ

[8] Count: 23
  REF token:     စွာ
  Predicted as:  များစွာ

[9] Count: 23
  REF token:     များ
  Predicted as:  သူများ

[10] Count: 22
  REF token:     စီးပွား
  Predicted as:  စီးပွားရေး

[11] Count: 22
  REF token:     ရေး
  Predicted as:  စီးပွားရေး

[12] Count: 21
  REF token:     ဖြစ်
  Predicted as:  ဖြစ်လာ

[13] Count: 21
  REF token:     လာ
  Predicted as:  ဖြစ်လာ

[1

## Summary

OpenFST command တွေနဲ့ myPOS version.3.0 dataset ကို သုံးပြီး WFST model ဆောက်ပြခဲ့တယ်။  
NLP experiment, Data science တွေရဲ့ အလုပ်တွေမို့လို့ ပုံမှန် data cleaning လုပ်ရတဲ့ အပိုင်းတွေလည်း လုပ်ပြခဲ့တယ် (ဥပမာ character segmentation, punctuation removing)။  
ပြီးတော့ ဒီတစ်ခေါက်က WFST model ဆောက်တာမို့လို့ symbol ဖိုင်တွေကို ဘယ်လိုဆောက်ရတယ် ဆိုတာကိုလည်း သင်ပေးခဲ့တယ်။  
FST model တွေကို သုံးပြီး အလုပ်လုပ်တဲ့အခါမှာ အရေးကြီးတဲ့ model optimization အပိုင်းကိုလည်း လုပ်ပြခဲ့တယ်။  
ဆရာ ကိုယ်တိုင်ရေးထားတဲ့ fst_decoder.py နဲ့ word segmentation လုပ်ပြခဲ့တယ်။  
WFST, FST မော်ဒယ်တွေမှာ ဖြစ်လေ့ဖြစ်ထ ရှိတဲ့ symbol ဖိုင်ထဲက သင်္ကတမတူလို့ decode မလုပ်ပေးနိုင်တာကိုလည်း လက်တွေ့ကြုံလာရတာမို့၊ အဲဒါကို ဘယ်လို ဖြေရှင်းရတယ်ဆိုတာကိုလည်း သင်ပြပေးခဲ့တယ်။  
နောက်ဆုံး အနေနဲ့ word segmentation အတွက် evaluation လုပ်တဲ့အခါမှာ သုံးတဲ့ မော်ဒယ်အပေါ်မူတည်ပြီး input ပေးလိုက်တဲ့ စာကြောင်းအရေအတွက်အတိုင်း ထွက်မလာတဲ့အခါမှာ ဘယ်လို evaluation လုပ်ရတယ်ဆိုတာကိုလည်း သရုပ်ပြပေးခဲ့တယ်။ ဆရာကိုယ်တိုင် အချိန်ပေး ပြင်ဆင်ထားတဲ့ ဒီလက်တွေ့လုပ်ရတဲ့ Lab ကနေ WFST ရဲ့ အသုံးဝင်ပုံကို အတိုင်းအတာတစ်ခုထိ နားလည်သွားပြီလို့ ယုံကြည်ပါတယ်။  :)    